# IRIS-Flower-classification

Этот проект представляет собой применение машинного обучения с программированием на Python на примере классификации цветов IRIS с использованием машинного обучения с инструментами scikit.
Здесь используются алгоритмы, которые представляют собой некоторые типы машинного обучения с учителем и без учителя.

[Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0) — набор данных для задачи классификации, на примере которого Рональд Фишер в 1936 году продемонстрировал работу разработанного им метода дискриминантного анализа. Иногда его также называют ирисами Андерсона, так как данные были собраны американским ботаником Эдгаром Андерсоном. Этот набор данных стал классическим и часто используется в литературе для иллюстрации работы различных статистических алгоритмов.

Ирисы Фишера состоят из данных о 150 экземплярах ириса, по 50 экземпляров из трёх видов — Ирис щетинистый (Iris setosa), Ирис виргинский (Iris virginica) и Ирис разноцветный (Iris versicolor). Для каждого экземпляра измерялись четыре характеристики (в сантиметрах):

- Длина наружной доли околоцветника (англ. sepal length);
- Ширина наружной доли околоцветника (англ. sepal width);
- Длина внутренней доли околоцветника (англ. petal length);
- Ширина внутренней доли околоцветника (англ. petal width).

На основании этого набора данных требуется построить правило классификации, определяющее вид растения по данным измерений. Это задача многоклассовой классификации, так как имеется три класса — три вида ириса.

Один из классов (Iris setosa) линейно-разделим от двух остальных.

В качестве задания вам предлагается повторить данные действия по классификации для наборов данных (можно выбрать, что больше понравится, но лучше сделать для всех)
- [User Knowledge Modeling Data Set](http://archive.ics.uci.edu/ml/datasets/User+Knowledge+Modeling)
- [Blood Transfusion Service Center Data Set](http://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center)
- [Somerville Happiness Survey Data Set](http://archive.ics.uci.edu/ml/datasets/Somerville+Happiness+Survey)
- [Wine Quality Data Set](https://archive.ics.uci.edu/ml/datasets/wine+quality)



In [2]:
# Подключаем библиотеки
import pandas as pd # Работа с набором данных
import numpy as np # Линейная алгебра
# Визуализация данных
import seaborn as sns
import matplotlib.pyplot as plt

# отключение информационных сообщений
from warnings import filterwarnings
filterwarnings(action='ignore')

# разделение выборки
from sklearn.model_selection import train_test_split
#метрики
from sklearn import metrics
#SVC - метод опорных векторов
from sklearn.svm import SVC

In [3]:
# читаем данные
url = 'https://raw.githubusercontent.com/AnnaA102/mo2025/refs/heads/main/semestr/Modeling_dataset.csv'
data = pd.read_csv(url)
data.head(5)

,STG,SCG,STR,LPR,PEG,UNS
0,0.00,0.00,0.00,0.00,0.00,very_low
1,0.08,0.08,0.10,0.24,0.90,High
2,0.06,0.06,0.05,0.25,0.33,Low
3,0.10,0.10,0.15,0.65,0.30,Middle
4,0.08,0.08,0.08,0.98,0.24,Low


In [4]:
# Разделим выборку на обучающую и тестовую
train, test = train_test_split(data, test_size = 0.4, stratify = data['UNS'], random_state = 42)

In [8]:
# Выделим признаки и результат
X_train = train[['STG','SCG','STR','LPR', 'PEG']]
y_train = train.UNS
X_test = test[['STG','SCG','STR','LPR', "PEG" ]]
y_test = test.UNS

fn = ['STG','SCG','STR','LPR', "PEG"]
cn = data['UNS'].unique()

## Метод опорных векторов

Рассмотрим подход к построению функции потерь,
основанный на максимизации зазора между классами.
Будем рассматривать линейные классификаторы вида
$$
    a(x) = sign (\langle w, x \rangle + b), \qquad w \in R^d, b \in R.
$$

Будем считать, что существуют такие параметры $w_*$ и $b_*$,
что соответствующий им классификатор $a(x)$ не допускает ни одной ошибки
на обучающей выборке.
В этом случае говорят, что выборка \emph{линейно разделима}.

Пусть задан некоторый классификатор $a(x) = sign (\langle w, x \rangle + b)$.
Заметим, что если одновременно умножить параметры $w$ и $b$
на одну и ту же положительную константу,
то классификатор не изменится.
Распорядимся этой свободой выбора и отнормируем параметры так, что
$$
    \min_{x \in X} | \langle w, x \rangle + b| = 1.
$$
Можно показать, что расстояние от произвольной точки $x_0 \in R^d$ до гиперплоскости,
определяемой данным классификатором, равно
$$
    \rho(x_0, a)
    =
    \frac{
        |\langle w, x \rangle + b|
    }{
        \|w\|
    }.
$$
Тогда расстояние от гиперплоскости до ближайшего объекта обучающей выборки равно
$$
    \min_{x \in X}
    \frac{
        |\langle w, x \rangle + b|
    }{
        \|w\|
    }
    =
    \frac{1}{\|w\|} \min_{x \in X} |\langle w, x \rangle + b|
    =
    \frac{1}{\|w\|}.
$$
Данная величина также называется \emph{отступом (margin)}.

Таким образом, если классификатор без ошибок разделяет обучающую выборку,
то ширина его разделяющей полосы равна $\frac{2}{\|w\|}$.
Известно, что максимизация ширины разделяющей полосы приводит
к повышению обобщающей способности классификатора \cite{mohri12foundations}.
Вспомним также, что на повышение обобщающей способности направлена и регуляризация,
которая штрафует большую норму весов --- а чем больше норма весов,
тем меньше ширина разделяющей полосы.

Итак, требуется построить классификатор, идеально разделяющий обучающую выборку,
и при этом имеющий максимальный отступ.
Запишем соответствующую оптимизационную задачу,
которая и будет определять метод опорных векторов для линейно разделимой выборки (hard margin support vector machine):
$$
    \left\{
        \begin{aligned}
            & \frac{1}{2} \|w\|^2 \to \min_{w, b} \\
            & y_i \left(
                \langle w, x_i \rangle + b
            \right) \geq 1, \quad i = 1, \dots, \ell.
        \end{aligned}
    \right.
$$
Здесь мы воспользовались тем, что линейный классификатор дает правильный ответ
на объекте $x_i$ тогда и только тогда, когда $y_i (\langle w, x_i \rangle + b) \geq 0$.
Из условия нормировки следует,
что $y_i (\langle w, x_i \rangle + b) \geq 1$.

В данной задаче функционал является строго выпуклым, а ограничения линейными,
поэтому сама задача является выпуклой и имеет единственное решение.
Более того, задача является квадратичной и может быть решена крайне эффективно.


In [9]:
# Метод опорных векторов, линейная модель
linear_svc = SVC(kernel='linear').fit(X_train, y_train)
y_pred=linear_svc.predict(X_test)

In [10]:
#Выполняем предсказание
y_pred=linear_svc.predict(X_test)

print('accuracy (точность) классификатора',metrics.accuracy_score(y_pred,y_test))

# матрица ошибок
print("матрица ошибок")
print(metrics.confusion_matrix(y_test, y_pred))

accuracy (точность) классификатора 0.8269230769230769
матрица ошибок
[[23  0  2  0]
 [ 0 33  0  0]
 [ 0  6 30  0]
 [ 0 10  0  0]]


In [12]:
# Использование построенной модели для предсказания класса
new_iris=[1.3, 0.4, 0.6, 0.5, 0.24]
y_new_iris=linear_svc.predict([new_iris])

print(y_new_iris)

['Low']


In [ ]:
# Опорные вектора
print(linear_svc.support_vectors_)

[[5.  3.  1.6 0.2]
 [4.8 3.4 1.9 0.2]
 [5.1 3.8 1.9 0.4]
 [5.7 2.6 3.5 1. ]
 [6.7 3.1 4.7 1.5]
 [6.8 2.8 4.8 1.4]
 [6.1 3.  4.6 1.4]
 [6.2 2.2 4.5 1.5]
 [5.9 3.2 4.8 1.8]
 [6.9 3.1 4.9 1.5]
 [6.3 2.5 4.9 1.5]
 [6.  2.7 5.1 1.6]
 [7.2 3.  5.8 1.6]
 [6.2 2.8 4.8 1.8]
 [5.6 2.8 4.9 2. ]
 [6.3 2.5 5.  1.9]
 [6.5 3.2 5.1 2. ]
 [5.9 3.  5.1 1.8]
 [6.  2.2 5.  1.5]]
